In [1]:
from itertools import product
import numpy as np

In [2]:
with open("input", "r") as f:
    droplet_coords = [tuple(int(coord) for coord in line.split(','))
                      for line in f.readlines()]

In [3]:
xs, ys, zs = zip(*droplet_coords)

min_x, max_x = min(xs), max(xs)
min_y, max_y = min(ys), max(ys)
min_z, max_z = min(zs), max(zs)

In [4]:
grid = np.zeros((max_x + 1, max_y + 1, max_z + 1), dtype=int)

for x, y, z in droplet_coords:
    grid[x, y, z] = 1

In [5]:
def neighbours(point):
    neighbour_coords = [(x, y, z)
                        for x, y, z in product(range(-1, 2), repeat=3)
                        if (sum(abs(_) for _ in (x, y, z)) == 1
                            and 0 <= point[0] + x <= max_x
                            and 0 <= point[1] + y <= max_y
                            and 0 <= point[2] + z <= max_z)]
    return [(point[0] + x, point[1] + y, point[2] + z)
            for (x, y, z) in neighbour_coords]

### Part 1

In [6]:
total_surface = sum(sum(grid[x, y, z] == 0
                        for x, y, z in neighbours([x0, y0, z0]))
                    + 6 - len(neighbours([x0, y0, z0]))
                    for x0 in range(min_x, max_x + 1)
                    for y0 in range(min_y, max_y + 1)
                    for z0 in range(min_z, max_z + 1)
                    if grid[x0, y0, z0] == 1)
total_surface

4314

### Part 2

In [7]:
def h(point_a, point_b):
    return sum(abs(coord_a - coord_b)
               for coord_a, coord_b in zip(point_a, point_b))

In [8]:
from heapq import heappop, heappush
from math import inf

def is_connected(start, goal, matrix, h):

    came_from = {}
    open_set = []
    g_score = {start: 0}
    f_score = {start: h(start, goal)}
    
    heappush(open_set, (f_score[start], start))
    
    x, y, z = start
    
    while len(open_set) > 0:
        _, current = heappop(open_set)
        x, y, z = current
        
        if current == goal:
            return True
        
        nbrs = [(nbr_x, nbr_y, nbr_z)
                for (nbr_x, nbr_y, nbr_z) in neighbours(current)
                if matrix[nbr_x, nbr_y, nbr_z] == 0]
        
        for neighbor in nbrs:
            tentative_g_score = g_score.get(current, inf) + 1

            if tentative_g_score < g_score.get(neighbor, inf):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + h(neighbor, goal)
                if (f_score[neighbor], neighbor) not in open_set:
                    heappush(open_set, (f_score[neighbor], neighbor))
    return False

In [9]:
zeros = [(x0, y0, z0)
         for x0 in range(min_x, max_x)
         for y0 in range(min_y, max_y)
         for z0 in range(min_z, max_z)
         if (grid[x0, y0, z0] == 0
             and any(grid[x, y, z] == 1
                     for x, y, z in neighbours((x0, y0, z0))))]

In [10]:
pocket_surface = sum(sum(grid[xn, yn, zn] for (xn, yn, zn) in neighbours((x0, y0, z0)))
                     for x0, y0, z0 in zeros
                     if not is_connected((x0, y0, z0), (0, 0, 0), grid, h))

In [11]:
total_surface - pocket_surface

2444